In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target

print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### iteration마다 일정한 batch 크기 만큼의 데이터를 전체 데이터 셋에서 random하게 가져와서 Mini-Batch GD를 수행

In [2]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    N = target_batch.shape[0]
    
    predicted_batch = w1*rm_batch + w2*lstat_batch + bias
    
    diff_batch = target_batch - predicted_batch
    
    bias_factors = np.ones((N, ))
    
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    return bias_update, w1_update, w2_update

In [3]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes)

[348 208  78 216 349 123 484 187 468 294 202 257 498  30  33 262 223 157
  49 190 435 152 166 246 173 480 209  45 179 177]


In [4]:
bostonDF['RM'].values[batch_indexes]

array([6.635, 6.064, 6.232, 5.888, 6.939, 5.856, 5.871, 6.782, 5.926,
       6.009, 7.61 , 8.704, 6.019, 5.713, 5.701, 8.398, 6.618, 6.943,
       5.602, 6.951, 6.629, 5.012, 7.929, 6.108, 6.416, 6.242, 5.344,
       5.682, 6.98 , 6.315])

In [7]:
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    
    np.random.seed = 2021
    w1 = np.zeros((1, ))
    w2 = np.zeros((1, ))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias: ', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    for i in range(iter_epochs):
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        
        if verbose:
            print('Epoch: ', i+1, '/', iter_epochs)
            predicted = w1*rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1: ', w1, 'w2: ', w2, 'bias: ', bias, 'loss: ', mse_loss)
    
    return w1, w2, bias

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print("##### 최종 w1, w2, bias #####")
print(w1, w2, bias)

최초 w1, w2, bias:  [0.] [0.] [0.]
Epoch:  1 / 5000
w1:  [0.23521101] w2:  [0.12061109] bias:  [0.42786667] loss:  565.9575807044436
Epoch:  2 / 5000
w1:  [0.45846384] w2:  [0.22967975] bias:  [0.85586869] loss:  540.8577091437886
Epoch:  3 / 5000
w1:  [0.67568459] w2:  [0.32401205] bias:  [1.24830355] loss:  518.1930475317992
Epoch:  4 / 5000
w1:  [0.8910504] w2:  [0.4163065] bias:  [1.63779649] loss:  496.2816609544484
Epoch:  5 / 5000
w1:  [1.11451707] w2:  [0.50602811] bias:  [2.0269043] loss:  474.7906012397036
Epoch:  6 / 5000
w1:  [1.34507172] w2:  [0.60011184] bias:  [2.42764271] loss:  453.22787114597776
Epoch:  7 / 5000
w1:  [1.55005737] w2:  [0.69423555] bias:  [2.8111979] loss:  433.4537674198836
Epoch:  8 / 5000
w1:  [1.72904025] w2:  [0.79337562] bias:  [3.15309706] loss:  416.2326745397335
Epoch:  9 / 5000
w1:  [1.93469358] w2:  [0.8692955] bias:  [3.51438635] loss:  398.40378506292416
Epoch:  10 / 5000
w1:  [2.13913746] w2:  [0.94998189] bias:  [3.87340915] loss:  381.124

Epoch:  202 / 5000
w1:  [11.51741459] w2:  [-0.05655709] bias:  [16.42880464] loss:  66.90830366039495
Epoch:  203 / 5000
w1:  [11.53460832] w2:  [-0.08448059] bias:  [16.42755674] loss:  66.8222189840524
Epoch:  204 / 5000
w1:  [11.5804959] w2:  [-0.08685976] bias:  [16.48014554] loss:  66.74612654832464
Epoch:  205 / 5000
w1:  [11.60314885] w2:  [-0.10474471] bias:  [16.49508776] loss:  66.6748591736002
Epoch:  206 / 5000
w1:  [11.60906916] w2:  [-0.14433074] bias:  [16.47655754] loss:  66.57839090983515
Epoch:  207 / 5000
w1:  [11.6324324] w2:  [-0.17254258] bias:  [16.47875544] loss:  66.48345109092408
Epoch:  208 / 5000
w1:  [11.65752191] w2:  [-0.19893708] bias:  [16.49218785] loss:  66.39002105985263
Epoch:  209 / 5000
w1:  [11.67397077] w2:  [-0.22194672] bias:  [16.49629402] loss:  66.31634631611604
Epoch:  210 / 5000
w1:  [11.67138125] w2:  [-0.25035132] bias:  [16.46567753] loss:  66.25772828772465
Epoch:  211 / 5000
w1:  [11.71441188] w2:  [-0.27187122] bias:  [16.49680308]

w1:  [13.98436034] w2:  [-4.3886887] bias:  [16.38243963] loss:  54.98505381674974
Epoch:  411 / 5000
w1:  [13.95630601] w2:  [-4.42422741] bias:  [16.32764043] loss:  54.98301333716499
Epoch:  412 / 5000
w1:  [13.9694018] w2:  [-4.43293338] bias:  [16.33779962] loss:  54.945584869480676
Epoch:  413 / 5000
w1:  [13.98189874] w2:  [-4.45498358] bias:  [16.33255013] loss:  54.893621368071365
Epoch:  414 / 5000
w1:  [14.00346498] w2:  [-4.46489205] bias:  [16.36519455] loss:  54.83361276742094
Epoch:  415 / 5000
w1:  [14.03430132] w2:  [-4.4731433] bias:  [16.39134815] loss:  54.77058471229573
Epoch:  416 / 5000
w1:  [14.05578661] w2:  [-4.48419232] bias:  [16.40788968] loss:  54.719606628772304
Epoch:  417 / 5000
w1:  [14.1060691] w2:  [-4.50730438] bias:  [16.45405614] loss:  54.60891303470487
Epoch:  418 / 5000
w1:  [14.08721678] w2:  [-4.54259344] bias:  [16.40347648] loss:  54.579119663109395
Epoch:  419 / 5000
w1:  [14.09131808] w2:  [-4.56471712] bias:  [16.39702077] loss:  54.5370

w1:  [16.13754708] w2:  [-7.88797632] bias:  [16.51496738] loss:  46.89758799389839
Epoch:  619 / 5000
w1:  [16.15980789] w2:  [-7.89352381] bias:  [16.53367446] loss:  46.871877334576894
Epoch:  620 / 5000
w1:  [16.15274925] w2:  [-7.90263665] bias:  [16.52055246] loss:  46.862348750288035
Epoch:  621 / 5000
w1:  [16.13235527] w2:  [-7.93069335] bias:  [16.48029086] loss:  46.83662371645847
Epoch:  622 / 5000
w1:  [16.10965304] w2:  [-7.96003148] bias:  [16.43062534] loss:  46.820003738885106
Epoch:  623 / 5000
w1:  [16.10914936] w2:  [-7.97927704] bias:  [16.4198742] loss:  46.794507534526154
Epoch:  624 / 5000
w1:  [16.13735506] w2:  [-7.99050167] bias:  [16.44242192] loss:  46.74630286192956
Epoch:  625 / 5000
w1:  [16.1443527] w2:  [-8.00324632] bias:  [16.43652371] loss:  46.721735865269366
Epoch:  626 / 5000
w1:  [16.16265692] w2:  [-8.01115123] bias:  [16.46306206] loss:  46.689222513458006
Epoch:  627 / 5000
w1:  [16.16375344] w2:  [-8.02863711] bias:  [16.45245838] loss:  46.

Epoch:  827 / 5000
w1:  [17.74571723] w2:  [-10.71582252] bias:  [16.50279199] loss:  41.65491556454407
Epoch:  828 / 5000
w1:  [17.76600228] w2:  [-10.72266248] bias:  [16.51988778] loss:  41.63224173393157
Epoch:  829 / 5000
w1:  [17.78357484] w2:  [-10.72860834] bias:  [16.53963983] loss:  41.614164504937186
Epoch:  830 / 5000
w1:  [17.76478693] w2:  [-10.75224555] bias:  [16.49698255] loss:  41.595205497383894
Epoch:  831 / 5000
w1:  [17.79672263] w2:  [-10.76300013] bias:  [16.52456266] loss:  41.559724676745596
Epoch:  832 / 5000
w1:  [17.79210682] w2:  [-10.78100569] bias:  [16.51759111] loss:  41.53964446149989
Epoch:  833 / 5000
w1:  [17.769511] w2:  [-10.80552819] bias:  [16.46777288] loss:  41.52720406029859
Epoch:  834 / 5000
w1:  [17.77513492] w2:  [-10.81905119] bias:  [16.46298627] loss:  41.50680056419588
Epoch:  835 / 5000
w1:  [17.77902389] w2:  [-10.83681841] bias:  [16.45245321] loss:  41.48345476666178
Epoch:  836 / 5000
w1:  [17.78312578] w2:  [-10.84935566] bias:

...

w1:  [25.41331368] w2:  [-23.11027935] bias:  [16.19976463] loss:  30.546374042211013
Epoch:  4035 / 5000
w1:  [25.42410908] w2:  [-23.11457363] bias:  [16.20135613] loss:  30.545244323130344
Epoch:  4036 / 5000
w1:  [25.40242407] w2:  [-23.12410169] bias:  [16.17078995] loss:  30.550515949123902
Epoch:  4037 / 5000
w1:  [25.42277816] w2:  [-23.11296084] bias:  [16.20336847] loss:  30.5452832296031
Epoch:  4038 / 5000
w1:  [25.40976034] w2:  [-23.12311141] bias:  [16.17699361] loss:  30.548770001101662
Epoch:  4039 / 5000
w1:  [25.40562771] w2:  [-23.11127352] bias:  [16.17944139] loss:  30.549086810992495
Epoch:  4040 / 5000
w1:  [25.40248928] w2:  [-23.11532511] bias:  [16.17104937] loss:  30.550527153366545
Epoch:  4041 / 5000
w1:  [25.39751147] w2:  [-23.12591932] bias:  [16.152256] loss:  30.55437932032216
Epoch:  4042 / 5000
w1:  [25.40041511] w2:  [-23.12000944] bias:  [16.16187511] loss:  30.55220768651806
Epoch:  4043 / 5000
w1:  [25.38435925] w2:  [-23.13163333] bias:  [16.13

w1:  [25.61077084] w2:  [-23.21577643] bias:  [16.30094101] loss:  30.550510877039212
Epoch:  4254 / 5000
w1:  [25.61848416] w2:  [-23.22093354] bias:  [16.30268139] loss:  30.55120040607503
Epoch:  4255 / 5000
w1:  [25.59720699] w2:  [-23.23616553] bias:  [16.2602849] loss:  30.5385115260366
Epoch:  4256 / 5000
w1:  [25.61356856] w2:  [-23.22909758] bias:  [16.28433783] loss:  30.5450750360793
Epoch:  4257 / 5000
w1:  [25.60660554] w2:  [-23.23116202] bias:  [16.26935466] loss:  30.541127509029845
Epoch:  4258 / 5000
w1:  [25.62244338] w2:  [-23.22286669] bias:  [16.29888068] loss:  30.550339607798552
Epoch:  4259 / 5000
w1:  [25.63396197] w2:  [-23.217153] bias:  [16.31784037] loss:  30.55820411102966
Epoch:  4260 / 5000
w1:  [25.65617286] w2:  [-23.22127577] bias:  [16.33476469] loss:  30.56698610717249
Epoch:  4261 / 5000
w1:  [25.6450108] w2:  [-23.22023747] bias:  [16.31944021] loss:  30.559855013235207
Epoch:  4262 / 5000
w1:  [25.63464284] w2:  [-23.23139227] bias:  [16.2937626

Epoch:  4474 / 5000
w1:  [25.60272826] w2:  [-23.17464246] bias:  [16.2392305] loss:  30.540692057558577
Epoch:  4475 / 5000
w1:  [25.6091558] w2:  [-23.16503147] bias:  [16.25833438] loss:  30.545663132865894
Epoch:  4476 / 5000
w1:  [25.61829673] w2:  [-23.16524333] bias:  [16.27490127] loss:  30.550345699548444
Epoch:  4477 / 5000
w1:  [25.61722063] w2:  [-23.17257141] bias:  [16.26681156] loss:  30.547403370551095
Epoch:  4478 / 5000
w1:  [25.61264147] w2:  [-23.16916079] bias:  [16.26164701] loss:  30.54621456305312
Epoch:  4479 / 5000
w1:  [25.60352082] w2:  [-23.17888575] bias:  [16.23882494] loss:  30.540282965630162
Epoch:  4480 / 5000
w1:  [25.61619857] w2:  [-23.16293157] bias:  [16.26353237] loss:  30.54761706709974
Epoch:  4481 / 5000
w1:  [25.60775066] w2:  [-23.16632687] bias:  [16.24718849] loss:  30.543165715941523
Epoch:  4482 / 5000
w1:  [25.58807736] w2:  [-23.17903035] bias:  [16.22125154] loss:  30.537363030476843
Epoch:  4483 / 5000
w1:  [25.59193678] w2:  [-23.1

w1:  [25.71420224] w2:  [-23.2976932] bias:  [16.17617839] loss:  30.52747199289514
Epoch:  4702 / 5000
w1:  [25.72314529] w2:  [-23.30204537] bias:  [16.17939694] loss:  30.527658137323993
Epoch:  4703 / 5000
w1:  [25.7228198] w2:  [-23.30394622] bias:  [16.18058253] loss:  30.52766011370821
Epoch:  4704 / 5000
w1:  [25.74015384] w2:  [-23.30857578] bias:  [16.19423402] loss:  30.529503849411917
Epoch:  4705 / 5000
w1:  [25.74126225] w2:  [-23.31634342] bias:  [16.19021214] loss:  30.528511936424525
Epoch:  4706 / 5000
w1:  [25.73274563] w2:  [-23.31651782] bias:  [16.17983282] loss:  30.527100419841595
Epoch:  4707 / 5000
w1:  [25.72220859] w2:  [-23.32209935] bias:  [16.15838674] loss:  30.52532813634526
Epoch:  4708 / 5000
w1:  [25.7028393] w2:  [-23.33328637] bias:  [16.12427223] loss:  30.526024209634986
Epoch:  4709 / 5000
w1:  [25.70840427] w2:  [-23.33472514] bias:  [16.14172537] loss:  30.52510687866396
Epoch:  4710 / 5000
w1:  [25.70061405] w2:  [-23.32978389] bias:  [16.132

w1:  [25.93527701] w2:  [-23.42309597] bias:  [16.23561829] loss:  30.549532037751575
Epoch:  4929 / 5000
w1:  [25.94855611] w2:  [-23.40490462] bias:  [16.27045431] loss:  30.56849630763166
Epoch:  4930 / 5000
w1:  [25.93216795] w2:  [-23.41650905] bias:  [16.233377] loss:  30.548996126361285
Epoch:  4931 / 5000
w1:  [25.92676959] w2:  [-23.41880226] bias:  [16.22604714] loss:  30.545382499665553
Epoch:  4932 / 5000
w1:  [25.92432503] w2:  [-23.42044948] bias:  [16.22149154] loss:  30.543364566091494
Epoch:  4933 / 5000
w1:  [25.93033973] w2:  [-23.42134216] bias:  [16.22285917] loss:  30.544601279348896
Epoch:  4934 / 5000
w1:  [25.92749886] w2:  [-23.4176797] bias:  [16.22442525] loss:  30.545083261252746
Epoch:  4935 / 5000
w1:  [25.92695677] w2:  [-23.42396995] bias:  [16.21822799] loss:  30.542348931650142
Epoch:  4936 / 5000
w1:  [25.93404123] w2:  [-23.42746818] bias:  [16.22921389] loss:  30.546608292898863
Epoch:  4937 / 5000
w1:  [25.94974912] w2:  [-23.43271848] bias:  [16.

In [9]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_BATCH_RANDOM
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,29.037708
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.573130
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.685845
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.463683
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.653486
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,28.167815
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.405680
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.840662
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.157328
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.335821
